In [1]:
%reset -f
#pip install tmdbsimple
import pandas as pd
import numpy as np
from numpy.random import shuffle
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
import tmdbsimple as tmdb
tmdb.API_KEY = 'abcd9827ce3fb2298891370ff0cd0e58'

In [2]:
def takeInfos(id):
    atores = []
    crews = []
    keywords = []
    sTitles = []
    sPopularity = []
    sMedia = []

    movie = tmdb.Movies(id)

    response = movie.info()
    popularity = round(movie.popularity)
    media = round(movie.vote_average)
    title = movie.title
    for ator in movie.credits()['cast']: atores.append(ator['name'])
    for crew in movie.credits()['crew']: crews.append(crew['name'])
    for keyword in movie.keywords()['keywords']: keywords.append(keyword['name'])
    for similar in movie.similar_movies()['results']: 
        sTitles.append(similar['title'])
        sPopularity.append(round(similar["popularity"]))
        sMedia.append(round(similar["vote_average"]))
    
    return title,popularity,media,atores,crews,keywords,sTitles,sPopularity,sMedia

def NormalizeCat(X): #Não esta sendo usado
    global perda
    i = 0
    n = len(X)
    limit = 300
    if len(X)>limit:perda+=1
    while(len(X)<limit):
        if i == n: i = 0
        X.append(X[i])
        i+=1
    while(len(X)>limit):
        X.pop()
    return X
        

In [3]:
class Encoder:
    def __init__(self,lista):
        self.Codex = []
        self.atualIndex = 1
        self.fit(lista)
        
    def fit(self,lista):
        for item in lista:
            onList = False
            for code in self.Codex:
                if item == code[0]: onList = True
            if not onList:
                self.Codex.append([item,self.atualIndex])
                self.atualIndex+=1
    
    def transform(self,lista):
        resposta = []
        for item in lista:
            for code in self.Codex:
                if item == code[0]: resposta.append(code[1])
        return resposta
    
    def fit_transform(self,lista):
        self.fit(lista)
        return self.transform(lista)
    
    def inverse_transform(self,lista):
        resposta = []
        for item in lista:
            for code in self.Codex:
                if item == code[1]: resposta.append(code[0])
        if len(lista)==1: return resposta[0]
        return resposta
    
    def show(self):
        print(self.Codex)
    
    def Feed(self,id):
        title,popularity,media,atores,crews,keywords,sTitles,sPopularity,sMedia = takeInfos(id)
        self.fit([title])
        self.fit(sTitles)
        self.fit(atores)
        self.fit(crews)
        self.fit(keywords)

        Atores = self.transform(atores)
        Crews = self.transform(crews)
        Keywords = self.transform(keywords)
        X = self.fit_transform(["P"+str(popularity)])
        if len(Crews)>0:X.append(Crews[0])
        else: X.append(X[-1])
        for i in range(3):
            if len(Atores)>(i): X.append(Atores[i])
            else: X.append(X[-1])

            if len(Keywords)>(i): X.append(Keywords[i])
            else: X.append(X[-1])
        y = [media]
        X = [X]
        return X,y

In [4]:
credits = pd.read_csv("tmdb_5000_credits.csv")
movies = pd.read_csv("tmdb_5000_movies.csv")

In [5]:
IDlist = list(movies["id"])
n = 3000
IDTrainList = IDlist[:n]
IDTestList = IDlist[n:]

In [6]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [7]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [34]:
#NÃO RODAR ISSO SE NÃO FOR MUUUITO NECESSARIO
Codex = Encoder(['M0',"M1","M2",'M3','M4','M5','M6','M7','M8','M9','M10'])
Naive = MultinomialNB()

In [ ]:
n = 100
i = 0
shuffle(IDTrainList)
for idd in IDTrainList:
    X.append(Codex.Feed(idd)[0])
    y.append(Codex.Feed(idd)[1])
    i += 1
    print(str(i)+"/"+str(n))
    if i == n:break
print("\nEnsinando...")
Naive.fit(X,y)

1/100
2/100
3/100
4/100
5/100
6/100
7/100
8/100
9/100
10/100
11/100
12/100
13/100
14/100
15/100
16/100
17/100
18/100
19/100
20/100
21/100
22/100
23/100
24/100
25/100
26/100
27/100
28/100
29/100
30/100
31/100
32/100
33/100
34/100
35/100
36/100
37/100
38/100
39/100
40/100
41/100
42/100


In [32]:
shuffle(IDTestList)
Matriz = [[0,0,0,0,0,0,0,0,0,0,0]]*11
for idd in IDTestList: 
    X,y = Codex.Feed(idd)
    saida = Naive.predict(X)
    
    Matriz[saida[0]][y[0]]+=1

    i += 1
    print(str(i)+"/"+str(n)+" [{0},{1}]".format(saida,y))
    if i == n:break

1/10[[7],[7]]
2/10[[7],[5]]
3/10[[7],[8]]
4/10[[7],[6]]
5/10[[7],[6]]
6/10[[7],[6]]
7/10[[7],[7]]
8/10[[7],[7]]
9/10[[7],[6]]
10/10[[7],[7]]


In [ ]:
search = tmdb.Search()
response = search.movie(query='The Man Who Invented')
for s in search.results: print(s['title'], s['id'], s['release_date'], s['popularity'], round(s['vote_average']))

In [29]:
for linha in Matriz:
    print(linha)

[0, 0, 0, 5, 7, 15, 30, 32, 9, 0, 2]
[0, 0, 0, 5, 7, 15, 30, 32, 9, 0, 2]
[0, 0, 0, 5, 7, 15, 30, 32, 9, 0, 2]
[0, 0, 0, 5, 7, 15, 30, 32, 9, 0, 2]
[0, 0, 0, 5, 7, 15, 30, 32, 9, 0, 2]
[0, 0, 0, 5, 7, 15, 30, 32, 9, 0, 2]
[0, 0, 0, 5, 7, 15, 30, 32, 9, 0, 2]
[0, 0, 0, 5, 7, 15, 30, 32, 9, 0, 2]
[0, 0, 0, 5, 7, 15, 30, 32, 9, 0, 2]
[0, 0, 0, 5, 7, 15, 30, 32, 9, 0, 2]
[0, 0, 0, 5, 7, 15, 30, 32, 9, 0, 2]


In [ ]:
print(saida[0])
print(y[0])